In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
file_paths = [f.path for f in os.scandir("data")]
file_paths

['data/change-visitors-grocery-stores.csv',
 'data/income-support-covid.csv',
 'data/international-travel-covid.csv',
 'data/public-gathering-rules-covid.csv',
 'data/internal-movement-covid.csv',
 'data/covid-contact-tracing.csv',
 'data/covid-vaccine-age.csv',
 'data/change-visitors-retail-recreation.csv',
 'data/public-campaigns-covid.csv',
 'data/workplace-closures-covid.csv',
 'data/debt-relief-covid.csv',
 'data/covid-vaccination-policy.csv',
 'data/face-covering-policies-covid.csv',
 'data/covid-19-testing-policy.csv',
 'data/stay-at-home-covid.csv',
 'data/school-closures-covid.csv',
 'data/public-transport-covid.csv',
 'data/youngest-age-covid-vaccination.csv',
 'data/public-events-covid.csv']

In [4]:
features = None

for file_path in file_paths:
    df = pd.read_csv(file_path)
    df = df[df.Entity == 'Israel']
    df = df.iloc[:,-2:]
    if type(features) is pd.DataFrame:
        features = features.merge(df, how='outer')
    else:
        features = df
    #df = df[['Day', 'testing_policy']]

#print(features)

features = features.fillna(-1)
need_casting = ['school_closures', 'workplace_closures']
for foo in need_casting:
    features[foo] = features[foo].astype(np.int64)
print(features.dtypes)
#print(features)

features = features.fillna(-1)
need_string = ['vaccine_eligibility_age']
for foo in need_string:
    features[foo] = features[foo].astype(str)
print(features.dtypes)
print(features)

Day                                 object
grocery_and_pharmacy               float64
income_support                       int64
international_travel_controls        int64
restriction_gatherings               int64
restrictions_internal_movements      int64
contact_tracing                      int64
vaccine_availability                 int64
retail_and_recreation              float64
public_information_campaigns         int64
workplace_closures                   int64
debt_relief                          int64
vaccination_policy                   int64
facial_coverings                     int64
testing_policy                       int64
stay_home_requirements               int64
school_closures                      int64
close_public_transport               int64
vaccine_eligibility_age             object
cancel_public_events                 int64
dtype: object
Day                                 object
grocery_and_pharmacy               float64
income_support                       int

In [5]:
#features['Day'] = pd.to_datetime(df['Day'], format="%Y-%m-%d")
#train = features.query("Day > '2020-03-02' and Day <= '2021-06-14'")
train = features[(features['Day'] > '2020-03-02') & (features['Day'] <= '2021-06-14')]
test = features[(features['Day'] > '2021-06-14') & (features['Day'] <= '2021-11-14')]
test = test.rename(columns={'Day':'date'})
train = train.rename(columns={'Day':'date'})
vaccine_eligibility_age_dictionary = {'5-15 yrs':1, '16-19 yrs':2,'35-39 yrs':3,'40-44 yrs':4,'45-49 yrs':5,'55-59 yrs':5,'60-64 yrs':6}

train['vaccine_eligibility_age'] = train['vaccine_eligibility_age'].apply(lambda x: vaccine_eligibility_age_dictionary.get(x, x))
test['vaccine_eligibility_age'] = test['vaccine_eligibility_age'].apply(lambda x: vaccine_eligibility_age_dictionary.get(x, x))

train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

,date,grocery_and_pharmacy,income_support,international_travel_controls,restriction_gatherings,restrictions_internal_movements,contact_tracing,vaccine_availability,retail_and_recreation,public_information_campaigns,workplace_closures,debt_relief,vaccination_policy,facial_coverings,testing_policy,stay_home_requirements,school_closures,close_public_transport,vaccine_eligibility_age,cancel_public_events
15,2020-03-03,6.143,0,3,0,0,0,0,7.571,2,0,0,0,0,1,0,0,0,-1,0
16,2020-03-04,7.286,0,3,1,0,0,0,7.571,2,0,0,0,0,1,0,0,0,-1,2
17,2020-03-05,8.714,0,3,1,0,0,0,7.143,2,0,0,0,0,1,0,0,0,-1,2
18,2020-03-06,9.000,0,3,1,0,0,0,5.429,2,0,0,0,0,1,0,0,0,-1,2
19,2020-03-07,12.857,0,3,1,0,0,0,5.857,2,0,0,0,0,1,0,0,0,-1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,2021-06-10,10.286,2,4,0,0,2,3,-5.000,2,1,1,5,2,2,0,1,0,1,0
480,2021-06-11,10.429,2,4,0,0,2,3,-4.857,2,1,1,5,2,2,0,1,0,1,0
481,2021-06-12,10.571,2,4,0,0,2,3,-4.714,2,1,1,5,2,2,0,1,0,1,0
482,2021-06-13,10.571,2,4,0,0,2,3,-4.571,2,1,1,5,2,2,0,1,0,1,0


In [8]:
'''
處理npi小於0資料
propressing < 0 value 

'''
df = pd.read_excel("./SEIR/Israel-covid19-2020-0221-2022-0228.xlsx", sheet_name = "工作表2")
dd = df.fillna(-1)
#dd[dd['daily_new_cases']==-1]
index_daily_new_cases = dd[dd['daily_new_cases']< 0].index

for i in range(len(index_daily_new_cases)):
    if dd.loc[index_daily_new_cases[i-1],'daily_new_cases'] >= 0:
        dd.loc[index_daily_new_cases[i],'daily_new_cases'] = dd.loc[index_daily_new_cases[i-1],'daily_new_cases']
    elif dd.loc[index_daily_new_cases[i+1],'daily_new_cases'] >= 0:
        dd.loc[index_daily_new_cases[i],'daily_new_cases'] = dd.loc[index_daily_new_cases[i+1],'daily_new_cases']
    else:
        dd.loc[index_daily_new_cases[i],'daily_new_cases'] = 0
    

dd[dd['new_deaths']< 0]
dd[dd['daily_recovered_cases']< 0]

index_infectious = dd[dd['infectious']< 0].index

for i in range(len(index_infectious)):
    if dd.loc[index_infectious[i-1],'infectious'] >= 0:
        dd.loc[index_infectious[i],'infectious'] = dd.loc[index_infectious[i-1],'infectious']
    #elif dd.loc[index_infectious[i+1],'infectious'] >= 0 and i+1 < len(dd):
    #    dd.loc[index_infectious[i],'infectious'] = dd.loc[index_infectious[i+1],'infectious']
    else:
        dd.loc[index_infectious[i],'infectious'] = 0
#dd.loc[index_daily_new_cases]
index_new_deaths = dd[dd['new_deaths']< 0].index

for i in range(len(index_new_deaths)):
    if dd.loc[index_new_deaths[i-1],'new_deaths'] >= 0:
        dd.loc[index_new_deaths[i],'new_deaths'] = dd.loc[index_new_deaths[i-1],'new_deaths']
    elif dd.loc[index_new_deaths[i+1],'new_deaths'] >= 0 and i+1 < len(dd):
        dd.loc[index_new_deaths[i],'new_deaths'] = dd.loc[index_new_deaths[i+1],'new_deaths']
    else:
        dd.loc[index_new_deaths[i],'new_deaths'] = 0


index_daily_recovered_cases = dd[dd['daily_recovered_cases']< 0].index

for i in range(len(index_daily_recovered_cases)):
    if dd.loc[index_daily_recovered_cases[i-1],'daily_recovered_cases'] >= 0:
        dd.loc[index_daily_recovered_cases[i],'daily_recovered_cases'] = dd.loc[index_daily_recovered_cases[i-1],'daily_recovered_cases']
    #elif dd.loc[index_daily_recovered_cases[i+1],'daily_recovered_cases'] >= 0:
    #    dd.loc[index_daily_recovered_cases[i],'daily_recovered_cases'] = dd.loc[index_daily_recovered_cases[i+1],'daily_recovered_cases']
    else:
        dd.loc[index_daily_recovered_cases[i],'daily_recovered_cases'] = 0
#check
dd[(dd.index.values < 0) == True]

train_dd = dd[(dd['date'] > '2020-03-02') & (dd['date'] <= '2021-06-14')]
test_dd = dd[(dd['date'] > '2021-06-14') & (dd['date'] <= '2021-11-14')]
dd_columns = ['date','daily_new_cases','daily_recovered_cases', 'new_deaths', 'infectious', 'cumulative_cases','cumulative_infectious', 'cumulative_recovered', 'cumulative_deaths']
train_dd = train_dd[dd_columns]
test_dd = train_dd[dd_columns]

train_dd.to_csv("train_dd.csv", index=False)
test_dd.to_csv("test_dd.csv", index=False)

#train['date'] = pd.to_datetime(train['date'], utc=True)
train['date'] = train['date'].astype('datetime64[ns]')
test['date'] = test['date'].astype('datetime64[ns]')
## merge two file
train_res = pd.merge(train_dd,train, on='date')
train_res.to_csv("train_res.csv", index=False)

test_res = pd.merge(test_dd,test, on='date')
test_res.to_csv("test_res.csv", index=False)


In [10]:
'''

畫npi資料

'''

import plotly.graph_objects as go

# Plotting
fig = go.Figure()

fig.add_trace(go.Scatter(x=dd['date']	, y=dd['infectious']	, mode='lines', name=f'infectious'))
fig.add_trace(go.Scatter(x=dd['date']	, y=dd['daily_new_cases']	, mode='lines', name=f'daily_new_cases'))
#fig.add_trace(go.Scatter(x=dd['date']	, y=dd['total_cases']	, mode='lines', name=f'total_cases'))
#fig.add_trace(go.Scatter(x=dd['date']	, y=dd['total_deaths']	, mode='lines', name=f'total_deaths'))
#fig.add_trace(go.Scatter(x=dd['date']	, y=dd['total_recovered']	, mode='lines', name=f'total_recovered'))
fig.add_trace(go.Scatter(x=dd['date']	, y=dd['reproduction_rate']	, mode='lines', name=f'reproduction_rate'))
#fig.add_trace(go.Scatter(x=dd['date']	, y=dd['cumulative_infectious']	, mode='lines', name=f'cumulative_infectious'))
fig.add_trace(go.Scatter(x=dd['date']	, y=dd['daily_recovered_cases']	, mode='lines', name=f'daily_recovered_cases'))
fig.add_trace(go.Scatter(x=dd['date']	, y=dd['new_deaths']	, mode='lines', name=f'new_deaths'))


# Styling
fig.update_layout(
    title='Israel-covid19-2020-0221-2022-0228',
    xaxis_title='時間',
    #yaxis_title='Value',
    template="plotly_dark",
)

fig.show()
